Bad form:



In [3]:
%pip install mediapipe opencv-python tensorflow pandas setuptools scikit-learn

You should consider upgrading via the '/Users/vikrantverma/Documents/mediapipe-form-tracker/py3-10/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import setuptools.dist
# from tensorflow.keras.layers import Dense, LSTM
# from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
import pandas as pd 
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image


mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
def calculate_angles(start, mid, end):
   start = np.array(start)
   mid = np.array(mid)
   end = np.array(end)

   radians = np.arctan2(end[1] - mid[1], end[0] - mid[0]) - np.arctan2(start[1] - mid[1], start[0] - mid[0])
   angle = np.abs(radians*180.0/np.pi)

   if angle > 180:
      angle = 360 - angle
   
   return angle


GATHER DATA, IF BICEP NOT FULLY STRETCHED THEN WE ARE CURLING

In [48]:
cap = cv2.VideoCapture('data/video/last video i hope.mp4')

counter = 0
stage = None
sequences = []
shoulders = []
labels = []

with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
   while cap.isOpened():
      ret, frame = cap.read()

      if not ret:
         break

      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      image.flags.writeable = False 

      result = pose.process(image)

      image.flags.writeable = True


      if result.pose_landmarks:
         mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

         # Extract relevant landmarks (elbow, shoulder, wrist)
         landmarks = result.pose_landmarks.landmark
         real = result.pose_world_landmarks.landmark

         l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]

         r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]

         elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
         
         real_elbow = [real[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  real[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                  real[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
         
         real_hip = [real[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                  real[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                  real[mp_pose.PoseLandmark.LEFT_HIP.value].z]
         
         r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]


         sequences.append([l_shoulder[2], elbow[2], l_shoulder[1], r_shoulder[1], real_elbow[2], real_hip[2]])
         # 0 == good
         # 1 == shoulder too high
         # 2 == shoulder too low
         # 3 == elbow too forward
         # 4 == 1 and 3
         # 5 == 2 and 3

         label = 0

         # if elbow is more negative then it is closer to camera, .1 is for buffer
         threshold = 0.015
         elbow_forward = abs(l_shoulder[0] - elbow[0]) > .1
         print(real_hip[2])
         print(real_elbow[2])
         shoulder_raised = l_shoulder[1] < r_shoulder[1] - threshold  # Left shoulder must be noticeably higher
         shoulder_dropped = l_shoulder[1] > r_shoulder[1] + threshold  # Left shoulder must be noticeably lower


         if shoulder_dropped and elbow_forward:
            label = 5
         elif shoulder_raised and elbow_forward:
            label = 4
         elif elbow_forward:
            label = 3
         elif shoulder_dropped:
            label = 2
         elif shoulder_raised:
            label = 1

         cv2.putText(
            image,
            str(label),
            (300,300),
            cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2, cv2.LINE_AA
         )


         labels.append(label)

      

      cv2.imshow('Strongr v1', image)

      key = cv2.waitKey(10)

      if key==ord('q'):
         cv2.destroyAllWindows()
         break

   cv2.waitKey(1)
   cap.release()
   cv2.destroyAllWindows()

# Convert to DataFrame for easier analysis
#print(sequences)
df = pd.DataFrame(sequences, columns=['l_shoulder_z', 'elbow_z', 'l_shoulder_y', 'r_shoulder_y', 'elbow_real_z', 'hip_real_z'])
df['label'] = labels

# Save to CSV
df.to_csv('bicep_curl_data.csv', index=False)





I0000 00:00:1724807675.809711 34041970 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1724807675.997492 34120683 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1724807676.008036 34120683 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/vikrantverma/Documents/mediapipe-form-tracker/py3-10/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


0.008004188537597656
-0.047470927238464355
0.007667307276278734
-0.06604007631540298
-0.0013423148775473237
-0.09366920590400696
-0.008888505399227142
-0.10010243952274323
-0.016653496772050858
-0.10663322359323502
-0.018029503524303436
-0.11809811741113663
-0.019837893545627594
-0.11943653970956802
-0.019469039514660835
-0.12347837537527084
-0.018716424703598022
-0.12566910684108734
-0.014805452898144722
-0.12142264097929001
-0.013863718137145042
-0.1197550892829895
-0.013459590263664722
-0.12145710736513138
-0.012787173502147198
-0.12061110883951187
-0.011659380979835987
-0.12018787860870361
-0.01146628987044096
-0.11404820531606674
-0.011251161806285381
-0.10365226119756699
-0.007098203990608454
-0.08923482894897461
-0.004096221178770065
-0.08503026515245438
-0.002904570195823908
-0.07216601073741913
-0.0019130901200696826
-0.07352571189403534
-0.0002473763597663492
-0.06615772843360901
0.0005777060287073255
-0.06708784401416779
0.0007435248699039221
-0.08889402449131012
0.002672805

In [50]:
X = df[['l_shoulder_z', 'elbow_z', 'l_shoulder_y', 'r_shoulder_y', 'elbow_real_z', 'hip_real_z']]  # Input features
y = df['label']  # Target label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

import joblib
joblib.dump(clf, 'BicepBro.pkl')

Accuracy: 0.9507042253521126
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97        52
           1       0.90      0.90      0.90        20
           2       0.95      1.00      0.97        19
           3       0.88      1.00      0.94        15
           4       0.94      0.94      0.94        33
           5       1.00      1.00      1.00         3

    accuracy                           0.95       142
   macro avg       0.95      0.96      0.95       142
weighted avg       0.95      0.95      0.95       142



['BicepBro.pkl']

In [8]:
[l_shoulder[2], elbow[2], l_shoulder[1], r_shoulder[1]]


[-0.8788228034973145,
 -0.35105669498443604,
 0.4195356070995331,
 0.3919207453727722]

TRAINING THE MODEL WITH DATA

In [9]:
# # Load the saved data (angles and labels)
# df = pd.read_csv('bicep_curl_data.csv')
# X = df[['angle']].values  # Assuming your angle column is named 'angle'
# y = df['label'].values

# # Optionally: Normalize the angle values for better model performance
# X = X / 180.0  # Assuming angles are in degrees (0 to 180)
# print(X)
# # Split the data into training and testing sets
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Build a simple neural network model
# model = Sequential()
# model.add(Dense(16, input_shape=(1,), activation='relu'))  # 1 input feature (angle)
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))  # Binary output (curl or not curl)

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# # Save the model
# model.save('bicep_curl_model.keras')







In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

print(X)


NameError: name 'model' is not defined